In [1]:
import xarray as xr
import numpy as np

In [105]:
lat = xr.DataArray([[0,1,2,0],[0,3,4,2],[0,1,2,3]])
long = xr.DataArray([[2,4,6,4],[2,8,6,4], [0,4,6,8]]).values.ravel()

shape = lat.shape

lat = lat.values.ravel()

In [120]:
unique = list(set([(y,x) for x,y in zip(lat, long)]))
unique.sort()
unique

[(0, 0), (2, 0), (4, 0), (4, 1), (4, 2), (6, 2), (6, 4), (8, 3)]

In [109]:
node = np.zeros(shape, dtype=int)
node

array([[0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0]])

In [113]:
node.reshape((-1,))[:] = [unique.index((y,x)) for x,y in zip(lat,long)]

In [114]:
node

array([[3, 7, 0, 1],
       [3, 6, 4, 5],
       [2, 7, 0, 6]])